In [6]:
import os, sys

base_path = os.path.join(os.getcwd(), "..")
print(f"Base Path: {base_path}")
sys.path.append(base_path)

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


In [7]:
# Load gym environment
import gym
from kube_sim_gym import *
from kube_sim_gym.envs.sim_kube_env import SimKubeEnv

In [8]:
from kube_hr_scheduler.scheduler.sim_hr_scheduler import SimHrScheduler
from kube_hr_scheduler.strategies.model.default import Model

In [9]:
import gym
import numpy as np
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data.types import Transitions

## Expert from dataset

In [10]:
# Test version
# data = np.genfromtxt('../dataset/data_expert.csv', delimiter=',', max_rows=1000000) 

# Final version
data = np.genfromtxt('../dataset/data_expert.csv', delimiter=',') 

In [11]:
# Split the data into obs, acts, infos, next_obs, and dones
obs = data[:, :12]
acts = data[:, 12]
infos = np.empty(len(data), dtype=dict)
next_obs = data[:, 13:25]
dones = data[:, 25].astype(bool)

In [12]:
transitions = Transitions(obs, acts, infos, next_obs, dones)

In [13]:
transitions[13]

{'obs': array([0.91, 0.85, 0.73, 0.76, 0.83, 0.75, 0.82, 0.94, 0.76, 0.84, 0.05,
        0.14]),
 'acts': 1.0,
 'infos': None,
 'next_obs': array([0.86, 0.71, 0.73, 0.76, 0.83, 0.75, 0.82, 0.94, 0.76, 0.84, 0.04,
        0.06]),
 'dones': False}

## Expert from newly trained model

In [9]:
# def train_expert():
#     print("Training a expert.")

#     expert = DQN(
#         policy='MlpPolicy',
#         env=env
#     )

#     expert.learn(10000)  # Note: change this to 100000 to train a decent expert.
#     return expert

In [10]:
# def sample_expert_transitions():
#     expert = train_expert()

#     print("Sampling expert transitions.")
#     rollouts = rollout.rollout(
#         expert,
#         DummyVecEnv([lambda: RolloutInfoWrapper(env)]),
#         rollout.make_sample_until(min_timesteps=None, min_episodes=50),
#         rng=rng,
#     )
#     return rollout.flatten_trajectories(rollouts)

In [20]:
# from stable_baselines3.common.evaluation import evaluate_policy

# def eval_unit(model, env, num_eval_episodes=2):
#     rews, lens = evaluate_policy(model, env, n_eval_episodes=num_eval_episodes, return_episode_rewards=True)
#     # Takes the mean of rews elements divided by lens elements
#     mean_rew = np.mean([rew / length for rew, length in zip(rews, lens)]).round(2)

#     # Takes the difference between the maximum and minimum of reward elements
#     std_rew = (np.max(rews) - np.min(rews)) / 2

#     mean_rew = round(mean_rew, 2)
#     std_rew = round(std_rew, 2)

#     return (mean_rew, std_rew)


# def eval_set(model):
#     eval_rur_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rur_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rur_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rur.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     eval_rbd1_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rbd1_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rbd1_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd1.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     eval_rbd2_env1 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
#     eval_rbd2_env2 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-10l-3m-1000p-10m_unbalanced.csv')
#     eval_rbd2_env3 = gym.make('SimKubeEnv-v0', reward_file='eval_rbd2.py', scenario_file='scenario-3l-10m-1000p-10m_unbalanced.csv')

#     rur1 = eval_unit(model, eval_rur_env1, num_eval_episodes=2)
#     rur2 = eval_unit(model, eval_rur_env2, num_eval_episodes=2)
#     rur3 = eval_unit(model, eval_rur_env3, num_eval_episodes=2)

#     print(rur1, rur2, rur3)

#     rbd11 = eval_unit(model, eval_rbd1_env1, num_eval_episodes=2)
#     rbd12 = eval_unit(model, eval_rbd1_env2, num_eval_episodes=2)
#     rbd13 = eval_unit(model, eval_rbd1_env3, num_eval_episodes=2)

#     print(rbd11, rbd12, rbd13)

#     rbd21 = eval_unit(model, eval_rbd2_env1, num_eval_episodes=2)
#     rbd22 = eval_unit(model, eval_rbd2_env2, num_eval_episodes=2)
#     rbd23 = eval_unit(model, eval_rbd2_env3, num_eval_episodes=2)

#     print(rbd21, rbd22, rbd23)
    

def evalnoma(eval_model):

    eval_env0 = gym.make("SimKubeEnv-v0", reward_file='train_step_3.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env1 = gym.make("SimKubeEnv-v0", reward_file='eval_rur.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env2 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
    eval_env3 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')

    step3 = evaluate_policy(eval_model, eval_env0, n_eval_episodes=1, return_episode_rewards=True)
    rur = evaluate_policy(eval_model, eval_env1, n_eval_episodes=1, return_episode_rewards=True)
    rbd1 = evaluate_policy(eval_model, eval_env2, n_eval_episodes=1, return_episode_rewards=True)
    rbd2 = evaluate_policy(eval_model, eval_env3, n_eval_episodes=1, return_episode_rewards=True)

    print(step3, rur, rbd1, rbd2)

## Student training

In [14]:
# Initialize environment without rendering
env = gym.make("SimKubeEnv-v0", reward_file='train_step_dynamic.py')
rng = np.random.default_rng(0)

Base Path: /Users/swkim/Documents/coding/thesis/PROMES_colab/notebook/..


In [15]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [ ]:
# eval_set(bc_trainer.policy)

In [16]:
# bc_trainer.train(log_interval=100000, n_batches=100000) # Test version 
bc_trainer.train(n_epochs=10, log_interval=100000) # Fianl version

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00179 |
|    entropy        | 1.79     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 88.5     |
|    loss           | 1.79     |
|    neglogp        | 1.79     |
|    prob_true_act  | 0.167    |
|    samples_so_far | 32       |
--------------------------------


99994batch [03:56, 451.85batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 100000    |
|    ent_loss       | -0.000229 |
|    entropy        | 0.229     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 1.3e+03   |
|    loss           | 0.128     |
|    neglogp        | 0.128     |
|    prob_true_act  | 0.906     |
|    samples_so_far | 3200032   |
---------------------------------


199971batch [08:34, 283.36batch/s]

...

899996batch [40:38, 324.21batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 900000    |
|    ent_loss       | -6.41e-05 |
|    entropy        | 0.0641    |
|    epoch          | 8         |
|    l2_loss        | 0         |
|    l2_norm        | 6.19e+03  |
|    loss           | 0.0554    |
|    neglogp        | 0.0555    |
|    prob_true_act  | 0.962     |
|    samples_so_far | 28800032  |
---------------------------------


999973batch [45:20, 402.39batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 1000000   |
|    ent_loss       | -0.000124 |
|    entropy        | 0.124     |
|    epoch          | 9         |
|    l2_loss        | 0         |
|    l2_norm        | 6.72e+03  |
|    loss           | 0.0827    |
|    neglogp        | 0.0828    |
|    prob_true_act  | 0.939     |
|    samples_so_far | 32000032  |
---------------------------------


1071450batch [48:18, 369.72batch/s]


In [ ]:
evalnoma(bc_trainer.policy)

In [17]:
from stable_baselines3.common.callbacks import EvalCallback
eval_env1 = gym.make("SimKubeEnv-v0", reward_file='eval_rur.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
eval_env2 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd1.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')
eval_env3 = gym.make("SimKubeEnv-v0", reward_file='eval_rbd2.py', scenario_file='scenario-5l-5m-1000p-10m_unbalanced.csv')

# Test version
# eval_callback1 = EvalCallback(eval_env1, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)
# eval_callback2 = EvalCallback(eval_env2, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)
# eval_callback3 = EvalCallback(eval_env3, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False)

# Final verison
eval_callback1 = EvalCallback(eval_env1, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rur")
eval_callback2 = EvalCallback(eval_env2, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rbd1")
eval_callback3 = EvalCallback(eval_env3, eval_freq=100000, n_eval_episodes=3, deterministic=True, render=False, log_path="results/poc_1/rbd2")

In [18]:
model1 = DQN('MlpPolicy', env, verbose=1)
model2 = DQN('MlpPolicy', env, verbose=1)
model3 = DQN('MlpPolicy', env, verbose=1)

model1.policy = bc_trainer.policy
model2.policy = bc_trainer.policy
model3.policy = bc_trainer.policy

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
# Test version
# model1.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model1.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback1
           )

KeyboardInterrupt: 

In [ ]:
# Test version
# model2.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model2.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback2
           )

In [ ]:
# Test version
# model3.learn(total_timesteps=5e5, 
#             log_interval=50, 
#             # progress_bar=True, 
#             callback=eval_callback1
#            )

# Final version
model3.learn(total_timesteps=1e6, 
            log_interval=50, 
            # progress_bar=True, 
            callback=eval_callback3
           )